In [1]:
# !pip install prophet
!pip install calendar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement calendar (from versions: none)
ERROR: No matching distribution found for calendar


In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime as dt

from prophet.serialize import model_to_json, model_from_json

import pandas as pd

from prophet import *
# from 

import pyarrow.parquet as pq


# from download import download

mpl.rcParams['figure.figsize'] = (24, 8)
mpl.rcParams['axes.grid'] = False


In [3]:
def drop_schema(df):
  drop_list = ['precip','windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover' ,'visibility',
        'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
        'icon', 'stations', 'name','feelslike', 'dew','precipprob', 'preciptype', 'snow', 'snowdepth']
  df = df.drop(drop_list , axis=1)
  return df


In [4]:
def resample_schema(df , period , type):
  df_id = df.set_index('datetime')
  if(type):
    df_id_sampled = df_id.resample(period).mean()
  else:
    df_id_sampled = df_id.resample(period).max()
  return df_id_sampled

In [5]:
def split(df , time_col ,split ,lower = None , upper = None):

  df['ds'] = df.index
  # - 
  df = df.rename({'heat_index' : 'y'}, axis = 'columns')
  split_dt = dt.datetime(split , 1 , 1 , 0 ,0)
  split_dt -= dt.timedelta(days=1)

  print("split at" , split_dt)
  if(lower == None):
    lower = df[time_col].iloc[0]
  if(upper == None):
    upper = df[time_col].iloc[-1]

  train = df[ (df[time_col] <= split_dt) ]
  test = df[(df[time_col] > split_dt) ]

  print(train.shape , test.shape)
  print(train.iloc[-1][time_col] , test.iloc[0][time_col])
  
  return train ,test

In [6]:
from sklearn.model_selection import train_test_split
def prophet_perp(df , pred_col):
  # - 
  df['ds'] = df.index
  # - 
  df = df.rename({pred_col : 'y'}, axis = 'columns')
  # - 
  train, test = train_test_split(df, test_size=0.2 , shuffle= False)

  # print(train.shape , test.shape)
  # print(train.iloc[-1][time_col] , test.iloc[0][time_col])
  
  return train , test
  

In [7]:
def prophet_train(train ):
  m = Prophet(
      
  )
  m.fit(train)
  print(m.params)
  return m


In [10]:
def train_model(city):
  CSV="content/{}.csv".format(city)
  df = pd.read_csv(CSV)
  df['datetime'] =  pd.to_datetime(df['datetime'], format='%Y%m%d %H:%M:%S')
  df = drop_schema(df) #function call
  T=(df['temp']*9/5)+32  
  df['temp']=T
  R=df['humidity']
  hi = -42.379 + 2.04901523*T + 10.14333127*R - 0.22475541*T*R - 6.83783*(10**-3)*(T*T) - 5.481717*(10**-2)*R*R + 1.22874*(10**-3)*T*T*R + 8.5282*(10**-4)*T*R*R - 1.99*(10**-6)*T*T*R*R
  df['heat_index'] = hi
  df = df.drop(['temp','humidity'] , axis=1)
  df_id_d = resample_schema(df , 'D' , 1) #function call
  df_id_d = df_id_d.loc[:'2022']
  train, test = split(df_id_d , 'ds', 2021) #function call
  train , test = prophet_perp(df_id_d , 'heat_index') #function call
  m = prophet_train(train) #function call
  path="content/{}_model.json".format(city)
  with open(path, 'w') as fout:
    fout.write(model_to_json(m))  # Save model



In [13]:
train_model('Adilabad')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/mbubqyc5.json


split at 2020-12-31 00:00:00
(3288, 2) (730, 2)
2020-12-31 00:00:00 2021-01-01 00:00:00


DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/szn4e0bd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=64391', 'data', 'file=/tmp/tmp138wjt2n/mbubqyc5.json', 'init=/tmp/tmp138wjt2n/szn4e0bd.json', 'output', 'file=/tmp/tmp138wjt2n/prophet_modeloom0_dc_/prophet_model-20230224055431.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:54:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:54:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


OrderedDict([('lp__', array([[9732.5]])), ('k', array([[-0.0265423]])), ('m', array([[0.81242]])), ('delta', array([[ 2.45568e-07,  2.25024e-03,  6.65607e-08, -2.49083e-01,
        -5.72612e-08,  3.14449e-01,  4.25200e-01, -6.49225e-08,
        -4.52262e-01, -7.20789e-02, -4.86280e-09,  4.30279e-01,
        -5.93741e-02, -6.53938e-01, -4.96939e-08,  5.10452e-04,
         4.83416e-01, -2.30820e-07, -1.46702e-07, -1.34206e-01,
        -1.16882e-01, -2.56526e-08,  2.12346e-01, -2.79037e-07,
        -1.63992e-01]])), ('sigma_obs', array([[0.0286772]])), ('beta', array([[ 5.41727e-02, -5.65168e-02, -2.38290e-02, -6.88189e-03,
         1.36618e-02,  1.73769e-02, -4.54472e-03, -5.08630e-04,
        -2.16087e-03,  5.09434e-03,  3.91850e-04, -5.37800e-03,
        -3.29249e-03,  2.19113e-03,  2.91733e-03,  2.09584e-03,
        -8.00167e-04, -9.13290e-06,  1.25992e-03,  1.42646e-03,
         1.96576e-04,  4.97895e-04, -2.38662e-04, -4.34311e-04,
        -1.17920e-04, -2.69671e-05]])), ('trend', a

In [14]:
train_model('Khammam')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/2ypsplbo.json


split at 2020-12-31 00:00:00
(3288, 2) (730, 2)
2020-12-31 00:00:00 2021-01-01 00:00:00


DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/7xdiulo8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77916', 'data', 'file=/tmp/tmp138wjt2n/2ypsplbo.json', 'init=/tmp/tmp138wjt2n/7xdiulo8.json', 'output', 'file=/tmp/tmp138wjt2n/prophet_modelib4dltbm/prophet_model-20230224055820.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:58:20 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:58:21 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


OrderedDict([('lp__', array([[9968.06]])), ('k', array([[-0.2123]])), ('m', array([[0.79883]])), ('delta', array([[-2.98325e-08,  2.52228e-06,  1.75948e-01,  3.62592e-02,
         5.60526e-05,  2.68041e-05,  2.46147e-01,  6.31147e-08,
        -2.61345e-01,  3.91648e-07,  2.72108e-02,  1.24932e-01,
        -5.07078e-06, -2.19503e-01, -2.04212e-06,  1.25579e-01,
         1.78576e-07, -7.71758e-02, -7.34086e-06,  3.78103e-08,
        -6.98349e-08,  2.77081e-02,  1.17285e-01, -6.02848e-09,
        -1.41151e-01]])), ('sigma_obs', array([[0.0270174]])), ('beta', array([[ 3.59310e-02, -9.35679e-02, -2.07317e-02, -2.60864e-02,
         8.77095e-03,  9.05248e-03, -4.49953e-05, -1.74021e-03,
        -3.67965e-03,  4.54681e-03,  2.47503e-03, -2.65250e-03,
        -1.57099e-03, -5.31507e-04,  2.72265e-03,  1.36031e-03,
        -2.43451e-03, -6.02022e-04,  1.56566e-04,  2.84398e-03,
         9.05907e-04,  9.51062e-04,  8.52267e-05, -4.50779e-04,
        -2.38850e-04,  3.66940e-05]])), ('trend', arr

In [15]:
train_model('Karimnagar')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/ffl5r72i.json


split at 2020-12-31 00:00:00
(3288, 2) (730, 2)
2020-12-31 00:00:00 2021-01-01 00:00:00


DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/wmlugygt.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22893', 'data', 'file=/tmp/tmp138wjt2n/ffl5r72i.json', 'init=/tmp/tmp138wjt2n/wmlugygt.json', 'output', 'file=/tmp/tmp138wjt2n/prophet_modelkrtl0ifx/prophet_model-20230224055844.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:58:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:58:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


OrderedDict([('lp__', array([[9909.]])), ('k', array([[-0.205309]])), ('m', array([[0.803894]])), ('delta', array([[-6.77852e-09,  1.19164e-01,  1.01593e-08, -2.15966e-05,
        -2.60246e-08,  2.68377e-02,  2.91117e-01,  8.20344e-07,
        -1.90744e-03, -3.38848e-02,  1.21814e-05,  5.42269e-02,
        -1.12200e-01, -2.61807e-01, -2.12040e-01, -1.20554e-07,
         3.38397e-01, -2.67698e-08,  9.53816e-08, -1.82968e-02,
        -2.78409e-03,  9.34396e-08,  4.16976e-02, -3.33510e-08,
        -8.82747e-02]])), ('sigma_obs', array([[0.0275171]])), ('beta', array([[ 4.43064e-02, -7.58033e-02, -2.23858e-02, -1.80084e-02,
         1.23817e-02,  1.68612e-02, -3.42091e-03, -6.20438e-04,
        -3.08142e-03,  5.14604e-03,  1.84379e-03, -4.29007e-03,
        -2.90241e-03,  1.56032e-03,  2.90899e-03,  3.03568e-03,
        -1.05230e-03, -4.96020e-04,  9.02163e-04,  2.08287e-03,
        -1.92414e-04,  6.80748e-04, -3.81442e-04, -3.27352e-04,
        -1.94456e-04,  1.09045e-05]])), ('trend', ar

In [16]:
train_model('Nizamabad')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/unzy9ylo.json


split at 2020-12-31 00:00:00
(3288, 2) (730, 2)
2020-12-31 00:00:00 2021-01-01 00:00:00


DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/m4h_cmmd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=18004', 'data', 'file=/tmp/tmp138wjt2n/unzy9ylo.json', 'init=/tmp/tmp138wjt2n/m4h_cmmd.json', 'output', 'file=/tmp/tmp138wjt2n/prophet_modelgts_zzqe/prophet_model-20230224055915.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:59:15 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:59:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


OrderedDict([('lp__', array([[9822.15]])), ('k', array([[-0.0602586]])), ('m', array([[0.815226]])), ('delta', array([[ 1.84686e-07,  2.12610e-02, -2.51324e-08, -2.95916e-01,
        -1.23852e-07,  4.02082e-01,  4.46191e-01,  6.88323e-08,
        -5.22468e-01, -2.83602e-07,  2.83443e-02,  3.72966e-01,
        -1.81948e-01, -5.21198e-01,  7.39771e-08,  8.52890e-02,
         2.79644e-01, -2.56923e-03,  4.15168e-08, -4.95506e-08,
        -3.09667e-06,  9.45402e-08,  7.82808e-10, -2.72828e-02,
        -8.93850e-02]])), ('sigma_obs', array([[0.0279744]])), ('beta', array([[ 5.44556e-02, -4.41746e-02, -2.30622e-02, -7.90682e-03,
         9.32815e-03,  1.64622e-02, -3.46257e-03, -2.20590e-03,
        -3.13673e-03,  5.36019e-03,  1.08036e-03, -4.45324e-03,
        -4.02282e-03,  6.18986e-04,  2.71632e-03,  2.87787e-03,
        -9.59841e-04, -3.15567e-04,  2.74584e-04,  2.09863e-03,
         1.20979e-04,  4.54306e-04, -3.61436e-04, -4.35014e-04,
        -6.38273e-05, -3.93513e-04]])), ('trend',

In [17]:
train_model('Warangal')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/aapmfazp.json


split at 2020-12-31 00:00:00
(3288, 2) (730, 2)
2020-12-31 00:00:00 2021-01-01 00:00:00


DEBUG:cmdstanpy:input tempfile: /tmp/tmp138wjt2n/x20hj6cl.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33197', 'data', 'file=/tmp/tmp138wjt2n/aapmfazp.json', 'init=/tmp/tmp138wjt2n/x20hj6cl.json', 'output', 'file=/tmp/tmp138wjt2n/prophet_modelqnb3_ym6/prophet_model-20230224055942.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:59:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:59:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


OrderedDict([('lp__', array([[10016.2]])), ('k', array([[-0.113979]])), ('m', array([[0.800257]])), ('delta', array([[-5.90116e-09,  1.47060e-07, -5.46052e-09, -9.41858e-02,
        -3.57251e-08, -4.99190e-08,  7.62133e-01,  2.00517e-08,
        -5.73149e-01, -2.64312e-06,  1.12231e-02,  5.29601e-01,
        -5.31168e-01, -3.19518e-01, -3.81149e-07,  3.45187e-01,
         4.45453e-04, -5.22532e-02,  4.81825e-08,  8.67422e-02,
         6.69193e-07,  2.51909e-07,  4.94199e-02, -7.01022e-08,
        -2.10542e-01]])), ('sigma_obs', array([[0.0262888]])), ('beta', array([[ 4.32848e-02, -6.77897e-02, -2.03462e-02, -1.36785e-02,
         8.20519e-03,  1.40157e-02, -1.88593e-03, -8.05715e-04,
        -3.23110e-03,  5.20423e-03,  2.29198e-03, -2.88900e-03,
        -2.30125e-03,  8.81873e-04,  2.64268e-03,  3.29183e-03,
        -1.25071e-03, -6.28613e-04, -6.86941e-05,  2.02509e-03,
         3.73512e-04,  4.93044e-04, -4.08949e-04, -5.14084e-04,
        -2.90180e-04, -6.33239e-07]])), ('trend', 